In [1]:
# importing libraries

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [2]:
# loading trainiter and testier

trainloader = torch.load('../../1_Data_Loader/Data_Loader_files/trainloader_V1_resnet18.pt')
testloader = torch.load('../../1_Data_Loader/Data_Loader_files/testloader_V1_resnet18.pt')

In [3]:
# setting GPU if it's available

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
# importing resnet101

model = models.resnet101(pretrained=True)
model.to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(nn.Linear(2048, 1000),
                                nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(1000, 200),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(200, 50),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(50, 3),
                                 nn.LogSoftmax(dim=1))

In [6]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

In [ ]:
# training loop



epochs = 100
steps = 0
running_loss = 0
print_every = 5
max_accuracy = 0
for epoch in range(epochs):
    for inputs, labels in trainloader:
        
        steps += 1
        # Move input and label tensors to the default device
        # inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    if accuracy >= max_accuracy:
                        max_accuracy = accuracy
                        torch.save(model.state_dict(), 'checkpoint.pth')
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()



Epoch 1/100.. Train loss: 1.975.. Test loss: 1.348.. Test accuracy: 0.390
Epoch 1/100.. Train loss: 1.016.. Test loss: 1.170.. Test accuracy: 0.390
Epoch 1/100.. Train loss: 0.932.. Test loss: 1.382.. Test accuracy: 0.387
Epoch 1/100.. Train loss: 0.815.. Test loss: 1.255.. Test accuracy: 0.387
Epoch 1/100.. Train loss: 0.939.. Test loss: 1.127.. Test accuracy: 0.390
Epoch 1/100.. Train loss: 0.947.. Test loss: 1.206.. Test accuracy: 0.394
Epoch 1/100.. Train loss: 0.862.. Test loss: 1.201.. Test accuracy: 0.398
Epoch 1/100.. Train loss: 0.808.. Test loss: 1.068.. Test accuracy: 0.529
Epoch 1/100.. Train loss: 0.523.. Test loss: 1.748.. Test accuracy: 0.545
Epoch 1/100.. Train loss: 0.548.. Test loss: 1.545.. Test accuracy: 0.566
Epoch 1/100.. Train loss: 0.603.. Test loss: 1.211.. Test accuracy: 0.532
Epoch 1/100.. Train loss: 0.634.. Test loss: 0.883.. Test accuracy: 0.547
Epoch 1/100.. Train loss: 0.741.. Test loss: 0.923.. Test accuracy: 0.581
Epoch 1/100.. Train loss: 0.471.. Test

In [ ]:
torch.save(model.state_dict(), "../Model_files/resnet101luca.pt")

In [ ]:
import torch.onnx
from torch.autograd import Variable

dummy_input = Variable(torch.randn(1, 3, 224, 224)) 

torch.onnx.export(model,
                  dummy_input,
                  "../Model_files/resnet101_luca.onnx",
                  export_params=True,
                  opset_version=10)